In [1]:
import sys

# IT'S 2025 WHY HASN'T SOMEONE FIXED THIS
if ".." not in sys.path:
    sys.path.insert(0, "..")
    
import pandas as pd
import numpy as np
import ast

import tasks.run_helper

In [2]:
df = pd.read_csv(
    "../data/100_annotators.csv",
    converters={"annot_personality_characteristics": ast.literal_eval},
)
df["toxicity"] = df.annotation.apply(lambda x: x[-1]).astype(int)
df["annot_politics"] = df.annot_personality_characteristics.apply(
    lambda x: x[0]
)
df.annot_age = pd.cut(df.annot_age, bins=4)
df.message_id = df.message_id.astype(str)
df["comment_key"] = df.conv_id + df.message_id

df = df.loc[
    :,
    [
        "conv_id",
        "message_id",
        "comment_key",
        "message",
        "toxicity",
        "annot_age",
        "annot_sex",
        "annot_sexual_orientation",
        "annot_demographic_group",
        "annot_current_employment",
        "annot_education_level",
        "annot_politics",
    ],
]
df = df.groupby(["conv_id", "message_id", "comment_key", "message"]).apply(
    lambda x: pd.Series(
        {
            col: x[col].tolist()
            for col in df.columns
            if col not in ["conv_id", "message_id", "comment_key", "message"]
        }
    ),
    include_groups=False,
)
df.head(1)

,,,,toxicity,annot_age,annot_sex,annot_sexual_orientation,annot_demographic_group,annot_current_employment,annot_education_level,annot_politics
conv_id,message_id,comment_key,message,,,,,,,,
250cc245-86c4-4e97-a8ce-31d5d6796f71,-1428289379724132680,250cc245-86c4-4e97-a8ce-31d5d6796f71-1428289379724132680,"well thats a no brainer hillary clinton gave huma abdein a security clearance when she has ties to a known terrorist group the muslim brotherhood, and her mother runs an anti american news paper in the middle east, debbie washed up crook shultz got the awan famaily security clearances and they were recent immigrants, had absolutely no IT experience and possible ties to terrorist groups in pakistan. its pretty clear our liberal ran government is a complete and total failure when it comes to national security. 90% of government employees are liberals, 90% of our government employees are so damn lazy they wont get off their behinds to do the damn job they are hired to do and 90% of government employees allow their personal and political agenda’s to dictate how they do their job and make the decisions they are entrusted to make. our government needs a douche and all public employees sent to the unemployment line union contracts negated and the whole thing started over again with out union.\n","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[(69.75, 88.0], (33.25, 51.5], (69.75, 88.0], ...","[female, non-binary, female, male, male, male,...","[homosexual, straight, straight, straight, bis...","[black, white, asian, other, asian, other, asi...","[blue-collar, white-collar, unemployed, white-...","[university, none, university, none, universit...","[right-wing conservative, right-wing conservat..."


In [3]:
tasks.run_helper.results(
    df,
    discussion_id_col="conv_id",
    sdb_column="annot_age",
    value_col="toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/4 [00:00<?, ?it/s]

Raw pvalues: {Interval(69.75, 88.0, closed='right'): 0.6321938616294029, Interval(33.25, 51.5, closed='right'): 0.29904416439627446, Interval(51.5, 69.75, closed='right'): 0.7329047855188413, Interval(14.927, 33.25, closed='right'): 0.022357943792713407}
Corrected pvalues:  [0.73290479 0.59808833 0.73290479 0.08943178]
Raw pvalues: {Interval(69.75, 88.0, closed='right'): 0.6154062772630423, Interval(33.25, 51.5, closed='right'): 0.12752080006929012, Interval(51.5, 69.75, closed='right'): 0.43005483616181134, Interval(14.927, 33.25, closed='right'): 0.7020611043601577}
Corrected pvalues:  [0.7020611 0.5100832 0.7020611 0.7020611]
Raw pvalues: {Interval(69.75, 88.0, closed='right'): 0.5177530452085369, Interval(33.25, 51.5, closed='right'): 0.39778857325513006, Interval(51.5, 69.75, closed='right'): 0.4748093049537405, Interval(14.927, 33.25, closed='right'): 0.6308447207714605}
Corrected pvalues:  [0.63084472 0.63084472 0.63084472 0.63084472]
Raw pvalues: {Interval(69.75, 88.0, closed='

,250cc245-86c4-4e97-a8ce-31d5d6796f71,2feee653-15e9-4c38-b7a2-96fe26f50c9f,434ddba0-393a-424d-9c14-4837d454a221,a004f55d-03b1-4058-919a-6e6531aaf32f
"(69.75, 88.0]",0.732905,0.702061,0.630845,0.622118
"(33.25, 51.5]",0.598088,0.510083,0.630845,0.622118
"(51.5, 69.75]",0.732905,0.702061,0.630845,0.622118
"(14.927, 33.25]",0.089432,0.702061,0.630845,0.622118


In [4]:
tasks.run_helper.results(
    df,
    discussion_id_col="conv_id",
    sdb_column="annot_sex",
    value_col="toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/4 [00:00<?, ?it/s]

Raw pvalues: {'female': 0.5372336151943604, 'male': 0.5378976176844691, 'non-binary': 0.05009892090125616}
Corrected pvalues:  [0.53789762 0.53789762 0.15029676]
Raw pvalues: {'female': 0.43085345273061093, 'male': 0.4453131433408172, 'non-binary': 0.7184425854046863}
Corrected pvalues:  [0.66796972 0.66796972 0.71844259]
Raw pvalues: {'female': 0.5498084438604749, 'male': 0.5423690118110134, 'non-binary': 0.0012448744607833039}
Corrected pvalues:  [0.54980844 0.54980844 0.00373462]
Raw pvalues: {'female': 0.504495863009689, 'male': 0.6054174356026071, 'non-binary': 0.05681359524455874}
Corrected pvalues:  [0.60541744 0.60541744 0.17044079]


,250cc245-86c4-4e97-a8ce-31d5d6796f71,2feee653-15e9-4c38-b7a2-96fe26f50c9f,434ddba0-393a-424d-9c14-4837d454a221,a004f55d-03b1-4058-919a-6e6531aaf32f
female,0.537898,0.667970,0.549808,0.605417
male,0.537898,0.667970,0.549808,0.605417
non-binary,0.150297,0.718443,0.003735,0.170441


In [5]:
tasks.run_helper.results(
    df,
    discussion_id_col="conv_id",
    sdb_column="annot_demographic_group",
    value_col="toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/4 [00:00<?, ?it/s]

Raw pvalues: {'white': 0.6989694468005274, 'other': 0.4817221637971175, 'black': 0.5387066924062521, 'asian': 0.3105067198097806}
Corrected pvalues:  [0.69896945 0.69896945 0.69896945 0.69896945]
Raw pvalues: {'white': 0.30540079367842776, 'other': 0.7036345407827194, 'black': 0.2694377299230437, 'asian': 0.5317118660385486}
Corrected pvalues:  [0.61080159 0.70363454 0.61080159 0.70363454]
Raw pvalues: {'white': 0.37296408481880916, 'other': 0.7389111408766595, 'black': 0.4693740417997644, 'asian': 0.11829573999805491}
Corrected pvalues:  [0.62583206 0.73891114 0.62583206 0.47318296]
Raw pvalues: {'white': 0.2710963069244775, 'other': 0.553144536568446, 'black': 0.43729100262366094, 'asian': 0.49366507566513856}
Corrected pvalues:  [0.55314454 0.55314454 0.55314454 0.55314454]


,250cc245-86c4-4e97-a8ce-31d5d6796f71,2feee653-15e9-4c38-b7a2-96fe26f50c9f,434ddba0-393a-424d-9c14-4837d454a221,a004f55d-03b1-4058-919a-6e6531aaf32f
white,0.698969,0.610802,0.625832,0.553145
other,0.698969,0.703635,0.738911,0.553145
black,0.698969,0.610802,0.625832,0.553145
asian,0.698969,0.703635,0.473183,0.553145


In [6]:
tasks.run_helper.results(
    df,
    discussion_id_col="conv_id",
    sdb_column="annot_sexual_orientation",
    value_col="toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/4 [00:00<?, ?it/s]

Raw pvalues: {'homosexual': 0.7229723867691271, 'bisexual': 0.46038831754275766, 'other': 0.6027122182937812, 'straight': 0.3310744094559591}
Corrected pvalues:  [0.72297239 0.72297239 0.72297239 0.72297239]
Raw pvalues: {'homosexual': 0.33822038238186536, 'bisexual': 0.527874748343447, 'other': 0.6672332696553611, 'straight': 0.45699980126992745}
Corrected pvalues:  [0.66723327 0.66723327 0.66723327 0.66723327]
Raw pvalues: {'homosexual': 0.475415914652514, 'bisexual': 0.17561311555253112, 'other': 0.7408091288679566, 'straight': 0.4248128268963588}
Corrected pvalues:  [0.63388789 0.63388789 0.74080913 0.63388789]
Raw pvalues: {'homosexual': 0.4112457194640599, 'bisexual': 0.2796503937415331, 'other': 0.6587285454310333, 'straight': 0.5037805882316718}
Corrected pvalues:  [0.65872855 0.65872855 0.65872855 0.65872855]


,250cc245-86c4-4e97-a8ce-31d5d6796f71,2feee653-15e9-4c38-b7a2-96fe26f50c9f,434ddba0-393a-424d-9c14-4837d454a221,a004f55d-03b1-4058-919a-6e6531aaf32f
homosexual,0.722972,0.667233,0.633888,0.658729
bisexual,0.722972,0.667233,0.633888,0.658729
other,0.722972,0.667233,0.740809,0.658729
straight,0.722972,0.667233,0.633888,0.658729


In [7]:
tasks.run_helper.results(
    df,
    discussion_id_col="conv_id",
    sdb_column="annot_current_employment",
    value_col="toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/4 [00:00<?, ?it/s]

Raw pvalues: {'white-collar': 0.4395351806811517, 'unemployed': 0.6108993092168001, 'blue-collar': 0.30683018937239837}
Corrected pvalues:  [0.61089931 0.61089931 0.61089931]
Raw pvalues: {'white-collar': 0.6324142955173637, 'unemployed': 0.46518689530160107, 'blue-collar': 0.22467447822568376}
Corrected pvalues:  [0.6324143 0.6324143 0.6324143]
Raw pvalues: {'white-collar': 0.4101329241645406, 'unemployed': 0.4817518028867248, 'blue-collar': 0.5876918091701144}
Corrected pvalues:  [0.58769181 0.58769181 0.58769181]
Raw pvalues: {'white-collar': 0.584704849015848, 'unemployed': 0.3487191257633963, 'blue-collar': 0.537956231896191}
Corrected pvalues:  [0.58470485 0.58470485 0.58470485]


,250cc245-86c4-4e97-a8ce-31d5d6796f71,2feee653-15e9-4c38-b7a2-96fe26f50c9f,434ddba0-393a-424d-9c14-4837d454a221,a004f55d-03b1-4058-919a-6e6531aaf32f
white-collar,0.610899,0.632414,0.587692,0.584705
unemployed,0.610899,0.632414,0.587692,0.584705
blue-collar,0.610899,0.632414,0.587692,0.584705


In [8]:
tasks.run_helper.results(
    df,
    discussion_id_col="conv_id",
    sdb_column="annot_education_level",
    value_col="toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/4 [00:00<?, ?it/s]

Raw pvalues: {'high-school': 0.42397591923632105, 'none': 0.6749919621507308, 'university': 0.30832282722655024}
Corrected pvalues:  [0.63596388 0.67499196 0.63596388]
Raw pvalues: {'high-school': 0.3488469054661603, 'none': 0.7408165698961342, 'university': 0.31129112152895727}
Corrected pvalues:  [0.52327036 0.74081657 0.52327036]
Raw pvalues: {'high-school': 0.5146743259903431, 'none': 0.4521522195303943, 'university': 0.4908854662626353}
Corrected pvalues:  [0.51467433 0.51467433 0.51467433]
Raw pvalues: {'high-school': 0.3537686297798564, 'none': 0.5189465891039086, 'university': 0.4879862013244564}
Corrected pvalues:  [0.51894659 0.51894659 0.51894659]


,250cc245-86c4-4e97-a8ce-31d5d6796f71,2feee653-15e9-4c38-b7a2-96fe26f50c9f,434ddba0-393a-424d-9c14-4837d454a221,a004f55d-03b1-4058-919a-6e6531aaf32f
high-school,0.635964,0.523270,0.514674,0.518947
none,0.674992,0.740817,0.514674,0.518947
university,0.635964,0.523270,0.514674,0.518947


In [9]:
tasks.run_helper.results(
    df,
    discussion_id_col="conv_id",
    sdb_column="annot_politics",
    value_col="toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/4 [00:00<?, ?it/s]

Raw pvalues: {'left-wing liberal': 0.5340987555470709, 'right-wing conservative': 0.22582414634104114, 'apolitical': 0.6702187784682212}
Corrected pvalues:  [0.67021878 0.67021878 0.67021878]
Raw pvalues: {'left-wing liberal': 0.2976584037048903, 'right-wing conservative': 0.7340595485753223, 'apolitical': 0.5564053457129263}
Corrected pvalues:  [0.73405955 0.73405955 0.73405955]
Raw pvalues: {'left-wing liberal': 0.3568149209053637, 'right-wing conservative': 0.5258194457464617, 'apolitical': 0.7262833955917236}
Corrected pvalues:  [0.7262834 0.7262834 0.7262834]
Raw pvalues: {'left-wing liberal': 0.4590490789310286, 'right-wing conservative': 0.22863995415895366, 'apolitical': 0.5652749023227774}
Corrected pvalues:  [0.5652749 0.5652749 0.5652749]


,250cc245-86c4-4e97-a8ce-31d5d6796f71,2feee653-15e9-4c38-b7a2-96fe26f50c9f,434ddba0-393a-424d-9c14-4837d454a221,a004f55d-03b1-4058-919a-6e6531aaf32f
left-wing liberal,0.670219,0.73406,0.726283,0.565275
right-wing conservative,0.670219,0.73406,0.726283,0.565275
apolitical,0.670219,0.73406,0.726283,0.565275
